# Main Classification

## 3 feature tiers:
- Model 1: Baseline (Crunchbase static only)
- Model 2: Macro (includes funding (timing) and Macro Econ)
- Model 3: Full (all features, w/ NLP)

## Classifiers:
- Logistic Regression
- Random Forest
- XGBoost
- CatBoost

## Setup

### Imports

In [ ]:
!pip install catboost

In [ ]:
# !pip install shap --no-binary shap xgboost scikit-learn

# !pip install shap lightgbm catboost joblib scikit-learn xgboost pandas numpy matplotlib tqdm --upgrade
# Restart runtime after installing!

import os
import shap
import numpy as np
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import time # For timing blocks


from sklearn.model_selection import RandomizedSearchCV, train_test_split # Use RandomizedSearchCV
from sklearn.metrics import (
   roc_auc_score, accuracy_score,
   precision_score, recall_score, f1_score, make_scorer
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC # Commented out SVM/MLP for brevity, can be added back
# from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.utils.validation import check_is_fitted

from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

# Ensure display options are set for pandas DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

from google.colab import drive, runtime
drive.mount('/content/drive', force_remount=True)

### Folders

In [ ]:
input_folder = '/content/drive/MyDrive/Senior/Thesis/Code/Data/Input Data/'
model_data_folder = input_folder + 'model_data'
output_folder = '/content/drive/MyDrive/Senior/Thesis/Code/Data/Output Data/Model'

# Path for metric checkpoint files (unique per run potentially)
run_timestamp = time.strftime("%Y%m%d-%H%M%S")
print(f"Run timestamp: {run_timestamp}")
model_results_checkpoints_path = os.path.join(output_folder, f"checkpoint_model_metrics_{run_timestamp}.csv")

# Check contents of folders
model_data_contents = os.listdir(model_data_folder)
print(model_data_contents)

### Load Dataset

In [ ]:
df = pd.read_csv(os.path.join(model_data_folder, 'merged_startup_data.csv'))

In [ ]:
print(df.shape)
print(df.info())
print()
print(df.head())

In [ ]:
null_summary = df.isnull().sum().sort_values(ascending=False)
print(null_summary.head(50))

## Define Feature Set + Preprocessing

### Config

In [ ]:
# ============================ CONFIG ============================

# --- Modeling Choices ---
USE_ITERATIVE_IMPUTER = False  # Set to True to use IterativeImputer (slower), False for SimpleImputer+indicator
CV_FOLDS = 3                   # Number of cross-validation folds for hyperparameter tuning
N_ITER_RANDOM_SEARCH = 30      # Number of parameter combinations to try in RandomizedSearchCV (adjust based on time)
FILTER_YEAR = True             # Set to False to include startups founded after 2020 (use with caution for long-term labels)
FILTER_YEAR_THRESHOLD = 2020

# --- Target Labels to Run ---
# TARGET_COLS_TO_RUN = ["success_label_5y", "success_label_10y"]
TARGET_COLS_TO_RUN = ["success_label_5y"]

# Optional: Filter by founding year based on config
if FILTER_YEAR:
    initial_rows = df.shape[0]
    df = df[df["founded_year"] <= FILTER_YEAR_THRESHOLD].copy()
    print(f"Filtered DataFrame to founded_year <= {FILTER_YEAR_THRESHOLD}. Shape: {df.shape} (removed {initial_rows - df.shape[0]} rows)")

# ============================ CLASSIFIERS ============================

# Define base classifiers with relevant default settings
# Note: random_state is set for reproducibility
# Note: class_weight/scale_pos_weight handled in train_evaluate_model
classifiers = {
    # "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1),
    # "LightGBM": LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1), # Suppress verbosity
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0, thread_count=-1), # Use all cores, suppress verbose output
    # "RandomForest": RandomForestClassifier(random_state=42, n_jobs=-1),
    # "LogisticRegression": LogisticRegression(max_iter=2000, random_state=42, solver='saga', n_jobs=-1) # Saga supports L1/L2 and is often good for larger datasets
}


# Define WIDER parameter grids for RandomizedSearchCV
# Using lists here, but consider scipy.stats distributions for a broader random search
param_grids_random = {
    # "XGBoost": {
    #     'n_estimators': [100, 200, 300, 400, 500],
    #     'max_depth': [3, 5, 7, 9, 11],
    #     'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    #     'subsample': [0.7, 0.8, 0.9, 1.0],
    #     'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    #     'gamma': [0, 0.1, 0.2, 0.3, 0.5],
    #     'reg_alpha': [0, 0.01, 0.1, 1],      # L1
    #     'reg_lambda': [0.5, 1, 1.5, 2, 3]       # L2
    # },
    #  "LightGBM": {
    #     'n_estimators': [100, 200, 300, 500, 700],
    #     'max_depth': [-1, 5, 10, 15, 20, 25], # -1 means no limit
    #     'learning_rate': [0.01, 0.05, 0.1, 0.15],
    #     'num_leaves': [31, 63, 127, 200, 255], # Typically < 2^max_depth
    #     'subsample': [0.7, 0.8, 0.9, 1.0],
    #     'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    #     'reg_alpha': [0, 0.01, 0.1, 0.5, 1], # L1
    #     'reg_lambda': [0, 0.01, 0.1, 0.5, 1]  # L2
    # },
    "CatBoost": {
        'iterations': [100, 200, 300, 500, 700],
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15],
        'l2_leaf_reg': [1, 3, 5, 7, 9], # L2 regularization
        'border_count': [32, 64, 128, 255], # Controls discretization
        'subsample': [0.7, 0.8, 0.9, 1.0] # Row sampling
    },
    # "RandomForest": {
    #     "n_estimators": [100, 200, 300, 400, 500],
    #     "max_depth": [10, 20, 30, 40, None], # Explore deeper trees
    #     "min_samples_split": [2, 5, 10, 15],
    #     "min_samples_leaf": [1, 3, 5, 7],
    #     "max_features": ['sqrt', 'log2'],
    #     "class_weight": ["balanced", "balanced_subsample"] # Try both balancing strategies
    # },
    # "LogisticRegression": {
    #     "C": [0.01, 0.1, 1, 10, 50, 100],
    #     "penalty": ["l1", "l2"]
    #     # solver='saga', class_weight='balanced' set in constructor
    # }
}

### Feature Engineering and Preprocessing

In [ ]:
# ============================ FEATURE ENGINEERING & PREPROCESSING ============================

# ---- Binarize Industry ----
print("Binarizing 'industry' column...")
# Ensure clean parsing of industry column
df["industry_split"] = df["industry"].fillna("").apply(lambda x: [s.strip() for s in x.split(",")])


# Define the 9 known industries
known_industries = [
   "Life Sciences", "Fintech", "Consumer Goods", "Technology",
   "Cleantech", "Transportation", "Media Entertainment and Gaming",
   "Telecom", "Real Estate"
]

# Binarize them using MultiLabelBinarizer with fixed class order
mlb = MultiLabelBinarizer(classes=known_industries)
industry_df = pd.DataFrame(
   mlb.fit_transform(df["industry_split"]),
   columns=[f"industry_{c.replace(' ', '_').replace('_and_', '_')}" for c in mlb.classes_] # Cleaner names
)

# Concatenate binary columns and drop the original/temp
df = pd.concat([df.drop(columns=["industry", "industry_split"]), industry_df], axis=1)
industry_columns = industry_df.columns.tolist()
print("Industry binarization complete.")

In [ ]:
# ---- STEP 1: Define Feature Sets ----

# STATIC_MODEL — Basic Crunchbase attributes only
features_static = [
    'founded_year', 'city', 'founder_count',
    'founder_gender_diversity', 'has_top_school_founder',
    'num_optimal_degrees', 'optimal_degree_ratio',
    'is_repeat_founder', 'founder_gender_missing', 'founder_degree_missing',
    'founder_school_missing', 'founder_desc_missing',
    'has_funding_data', 'num_disclosed_rounds', 'has_disclosed_funding',
    'first_funding_amount_bucket', 'is_startup_hub',
    'cohort_funding_density', 'investor_count', 'has_known_investor'
] + industry_columns

# MACRO_MODEL — Add funding + macroeconomic/timing features
features_macro = features_static + [
    # Timing & funding
    'age_at_first_funding', 'first_funding_delay', 'early_series_count',
    'avg_time_to_early_round_months', 'avg_time_between_rounds',
    'burn_rate', 'funding_velocity',

    # Economic & market indicators
    'gdp_growth_avg_15m', 'gdp_growth_delta_3m', 'interest_rate_fed_funds_avg_15m',
    'interest_rate_fed_funds_delta_3m', 'fed_funds_rate_latest',
    'yield_curve_10y_2y_avg_15m', 'yield_curve_inversion_flag',
    'unemployment_rate_avg_15m', 'cpi_inflation_avg_15m',
    'consumer_sentiment_avg_15m', 'consumer_sentiment_z_latest',
    'vix_index_max_15m', 'vix_spike_flag', 'vix_latest',
    'sp500_price_change_3m', 'sp500_volatility_3m', 'sp500_momentum_latest',
    'avg_etf_return_3m', 'avg_etf_volatility_3m', 'avg_etf_momentum_latest',
    'avg_etf_golden_cross_flag', 'avg_etf_ma50_to_price_ratio'
]

# FULL_MODEL — All above + LLM and sentiment features
features_full = features_macro + [
    'org_desc_sentiment_finbert', 'founder_desc_sentiment_finbert',
    'org_desc_sim_exemplar', 'founder_desc_sim_exemplar',
    'llm_founder_score', 'industry_outlook_sentiment_finbert',
    'industry_timing_sentiment_finbert',
    'llm_outlook_align_score_avg', 'llm_outlook_align_score_binned'
]

# ---- STEP 2: Store in dictionary for easier looped modeling ----
model_feature_sets = {
    "STATIC_MODEL": features_static,
    # "MACRO_MODEL": features_macro,
    "FULL_MODEL": features_full,
}

print("Feature sets defined.")
print(f"STATIC features: {len(features_static)}")
# print(f"MACRO features: {len(features_macro)}")
print(f"FULL features: {len(features_full)}")

## Train/Test Split

In [ ]:
# ============================ DATA PREP FUNCTION ============================
def prepare_data_for_model(df, feature_list, target_col, test_size=0.2, random_state=42):
    """
    Prepares data for a specific model tier and target.
    Includes splitting, handling inf/all-NaN columns, imputation, scaling, and encoding.


    Args:
        df (pd.DataFrame): The input DataFrame.
        feature_list (list): List of feature names to use.
        target_col (str): The name of the target column.
        test_size (float): Proportion of data for the test set.
        random_state (int): Random seed for reproducibility.


    Returns:
        tuple: X_train_transformed, X_test_transformed, y_train, y_test, fitted_preprocessor
    """
    print(f"Preparing data for target '{target_col}' with {len(feature_list)} features...")
    df_model = df.dropna(subset=[target_col])
    X = df_model[feature_list].copy()
    y = df_model[target_col]


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


    # --- Clean infinite values and drop columns that became all NaN after split ---
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_train = X_train.dropna(axis=1, how="all")
    X_test = X_test[X_train.columns]  # Align test columns to train columns


    # --- Identify Feature Types ---
    categorical_features = list(X_train.select_dtypes(include=['object', 'category']).columns)
    # Ensure 'city' and 'first_funding_amount_bucket' are treated as categorical if they exist
    manual_cats = ['city', 'first_funding_amount_bucket']
    for cat in manual_cats:
        if cat in X_train.columns and cat not in categorical_features:
             categorical_features.append(cat)

    numerical_features = list(X_train.select_dtypes(include=np.number).columns)
    # Remove any manual cats that might have been inferred as numeric (e.g., if encoded previously)
    numerical_features = [f for f in numerical_features if f not in categorical_features]

    print(f"Identified {len(numerical_features)} numerical features.")
    print(f"Identified {len(categorical_features)} categorical features.")


    # --- Define Preprocessing Steps ---
    if USE_ITERATIVE_IMPUTER:
        print("Using IterativeImputer for numerical features.")
        numeric_imputer = IterativeImputer(max_iter=10, random_state=random_state) # Faster iteration count
    else:
        print("Using SimpleImputer (median) with missing indicators for numerical features.")
        # Add indicator for SimpleImputer is crucial for tree models with high missingness
        numeric_imputer = SimpleImputer(strategy="median", add_indicator=True)


    # Using most_frequent for categorical is standard
    categorical_imputer = SimpleImputer(strategy="most_frequent")

    numeric_transformer = Pipeline([
        ("imputer", numeric_imputer),
        ("scaler", StandardScaler()) # Scale after imputation
    ])
    categorical_transformer = Pipeline([
        ("imputer", categorical_imputer),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) # Use sparse=False for easier SHAP handling later if needed
    ])

    # --- Create ColumnTransformer ---
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numerical_features),
            ("cat", categorical_transformer, categorical_features)
        ],
        remainder='passthrough' # Keep any columns not specified (shouldn't be any here)
    )


    # --- Fit and Transform ---
    print("Fitting preprocessor and transforming data...")
    preprocessor.fit(X_train)
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    print("Data transformation complete.")

    # --- Get Feature Names After Transformation ---
    # This is essential for interpreting SHAP plots correctly
    try:
        feature_names_out = preprocessor.get_feature_names_out()
    except Exception as e:
        print(f"Warning: Could not automatically get feature names. Error: {e}. Manual construction might be needed.")
        feature_names_out = None # Handle this case downstream


    return X_train_transformed, X_test_transformed, y_train.astype(int), y_test.astype(int), preprocessor, feature_names_out

## Training, Eval, SHAP

### Functions

In [ ]:
# ============================ MODEL TRAINING & EVALUATION ============================

def train_evaluate_model(X_train, X_test, y_train, y_test, model_name, classifier_config, param_grid_config, cv_folds, n_iter_search, random_state=42):
    """
    Trains a model using RandomizedSearchCV (if grid provided), handles class imbalance, and evaluates.


    Args:
        X_train, X_test, y_train, y_test: Data splits.
        model_name (str): Name of the model (key in classifiers).
        classifier_config (dict): Dictionary of base classifier instances.
        param_grid_config (dict): Dictionary of parameter grids for RandomizedSearchCV.
        cv_folds (int): Number of CV folds for tuning.
        n_iter_search (int): Number of iterations for RandomizedSearchCV.
        random_state (int): Random seed.


    Returns:
        tuple: best_model, auc, acc, prec, rec, f1, best_params
    """
    start_time = time.time()
    print(f"  Training {model_name}...")
    model = classifier_config[model_name] # Get a fresh instance


    # --- Handle Class Imbalance ---
    scale_pos_weight = None
    # Calculate only if model supports it and isn't already balanced by class_weight
    if model_name in ["XGBoost", "CatBoost", "LightGBM"] and 'class_weight' not in model.get_params():
        neg, pos = np.bincount(y_train)
        if pos > 0: scale_pos_weight = neg / pos
        print(f"Calculated scale_pos_weight for {model_name}: {scale_pos_weight:.2f}")


    # Apply scale_pos_weight or class_weight to the base model *before* tuning
    try:
        if scale_pos_weight is not None:
             if model_name == "XGBoost": model.set_params(scale_pos_weight=scale_pos_weight)
             if model_name == "CatBoost": model.set_params(scale_pos_weight=scale_pos_weight)
             if model_name == "LightGBM": model.set_params(scale_pos_weight=scale_pos_weight) # or is_unbalance=True / class_weight='balanced'
        # class_weight='balanced'/'balanced_subsample' already set in classifier definition for RF/LR
    except Exception as e:
        print(f"    Warning: Could not set imbalance parameter for {model_name}. Error: {e}")


    # --- Hyperparameter Tuning ---
    if model_name in param_grid_config:
        print(f"Running RandomizedSearchCV with n_iter={n_iter_search}, cv={cv_folds}...")
        random_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid_config[model_name],
            n_iter=n_iter_search,
            cv=cv_folds,
            scoring='roc_auc', # Optimize for AUC during tuning
            verbose=0,
            random_state=random_state,
            n_jobs=-1 # Use all cores
        )
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        best_params = random_search.best_params_
        print(f"Best params found: {best_params}")
    else:
        # Fit default model if no grid (no need for logisitic regression - taking too long)
        print("No param grid defined, fitting default model.")
        best_model = model.fit(X_train, y_train)
        best_params = model.get_params()


    # --- Evaluation ---
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0) # Handle zero division
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    end_time = time.time()
    print(f"Training completed in {(end_time - start_time):.2f} seconds. AUC: {auc:.4f}, Acc: {acc:.4f}")

    return best_model, auc, acc, prec, rec, f1, best_params


In [ ]:
# ============================ SHAP ANALYSIS ============================

def get_ranked_shap_features(model, X_train_transformed, feature_names):
    """
    Calculates SHAP values and returns a DataFrame of features ranked by mean absolute SHAP value.


    Args:
        model: The trained model instance.
        X_train_transformed: The preprocessed training data used to train the model.
        feature_names (list): List of feature names corresponding to X_train_transformed columns.


    Returns:
        pd.DataFrame: DataFrame with 'feature' and 'mean_abs_shap', sorted descending.
                      Returns None if SHAP calculation fails.
    """
    print("Calculating SHAP values for feature importance...")
    start_time = time.time()
    shap_values_for_importance = None
    explainer = None

    try:
        # Choose appropriate explainer
        if isinstance(model, (XGBClassifier, LGBMClassifier, CatBoostClassifier, RandomForestClassifier)):
            explainer = shap.TreeExplainer(model)
            shap_values_calc = explainer.shap_values(X_train_transformed)
            # Handle potential list output for binary classification in some SHAP/model versions
            if isinstance(shap_values_calc, list) and len(shap_values_calc) == 2:
                shap_values_for_importance = shap_values_calc[1] # SHAP values for the positive class
            else:
                shap_values_for_importance = shap_values_calc # Assume it returns values for positive class

        elif isinstance(model, LogisticRegression):
            # LinearExplainer is efficient for linear models
            explainer = shap.LinearExplainer(model, X_train_transformed)
            shap_values_for_importance = explainer.shap_values(X_train_transformed)

        else: # Fallback for SVM, MLP etc. - can be slow!
            print("Warning: Using KernelExplainer for importance. Subsampling data...")
            # Sample background data for KernelExplainer efficiency
            X_train_summary = shap.sample(X_train_transformed, 100 if X_train_transformed.shape[0] > 100 else X_train_transformed.shape[0])
            explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)
            # Calculate SHAP for a subset of X_train for feasibility
            shap_values_list = explainer.shap_values(X_train_transformed[:500], nsamples='auto') # Explain first 500 samples
            if isinstance(shap_values_list, list) and len(shap_values_list) == 2:
                shap_values_for_importance = shap_values_list[1] # SHAP values for class 1
            else:
                 shap_values_for_importance = shap_values_list

        if shap_values_for_importance is None:
             print("Error: Could not obtain SHAP values.")
             return None

        # Ensure correct shape (samples, features)
        if len(shap_values_for_importance.shape) > 2:
            shap_values_for_importance = shap_values_for_importance[:, :, 1]

        # Calculate mean absolute SHAP value
        mean_abs_shap = np.abs(shap_values_for_importance).mean(axis=0)

        # Create DataFrame
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'mean_abs_shap': mean_abs_shap
        })

        # Sort by importance
        feature_importance_df = feature_importance_df.sort_values(by='mean_abs_shap', ascending=False).reset_index(drop=True)

        end_time = time.time()
        print(f"  SHAP value calculation completed in {(end_time - start_time):.2f} seconds.")
        return feature_importance_df

    except Exception as e:
        print(f"  SHAP calculation failed: {e}")
        return None

# --- SHAP Plotting Function (Optional - can be integrated or separate) ---
def plot_shap_summary(model, X_train_transformed, feature_names, title):
    """Generates and shows a SHAP beeswarm plot."""
    print(f"\nGenerating SHAP Summary Plot for {title}")
    try:
         # Re-calculate SHAP specifically for plotting if needed, or use values if passed
         explainer = None
         if isinstance(model, (XGBClassifier, LGBMClassifier, CatBoostClassifier, RandomForestClassifier)):
             explainer = shap.TreeExplainer(model)
             shap_values_plot = explainer(X_train_transformed) # Use explainer.__call__ for Explanation object
         elif isinstance(model, LogisticRegression):
              explainer = shap.LinearExplainer(model, X_train_transformed)
              shap_values_plot = explainer(X_train_transformed)
         else:
             print(f"Warning: Using KernelExplainer for plot {title}. This can be slow.")
             X_train_summary = shap.sample(X_train_transformed, 100)
             explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)
             shap_values_list = explainer.shap_values(X_train_transformed[:200], nsamples='auto') # Explain fewer for plot
             # Need to construct Explanation object manually for KernelExplainer plot
             shap_values_plot = shap.Explanation(values=shap_values_list[1], # Values for positive class
                                                  base_values=explainer.expected_value[1],
                                                  data=X_train_transformed[:200],
                                                  feature_names=feature_names)

         shap.plots.beeswarm(shap_values_plot, max_display=30, show=False)
         plt.title(title)
         plt.tight_layout()
         plt.show()

    except Exception as e:
         print(f"SHAP plot generation failed for {title}: {e}")

In [ ]:
# ============================ RETRAINING PIPELINE ============================
def retrain_with_top_features(df, target_col, original_results_df, feature_importance_df, k_features, model_tier_to_retrain="FULL_MODEL", random_state=42):
    """
    Retrains the best model from a specific tier using only the top k features identified by SHAP.


    Args:
        df (pd.DataFrame): The original DataFrame (before splitting/preprocessing).
        target_col (str): Name of the target variable column.
        original_results_df (pd.DataFrame): DataFrame containing results from the full_pipeline run.
        feature_importance_df (pd.DataFrame): Sorted DataFrame from get_ranked_shap_features.
        k_features (int): Number of top features to use for retraining.
        model_tier_to_retrain (str): The feature tier whose best model we are retraining.
        random_state (int): Random seed.


    Returns:
        dict: Dictionary containing metrics and info for the retrained model, or None if fails.
    """
    print(f"\n=== RETRAINING WITH TOP {k_features} FEATURES for target '{target_col}' ===")


    if feature_importance_df is None or k_features > feature_importance_df.shape[0]:
        print(f"  Error: Invalid feature importance data or k ({k_features}) > total features.")
        return None


    # --- Identify Best Original Model and its Params ---
    tier_results = original_results_df[original_results_df['Feature_Tier'] == model_tier_to_retrain]
    if tier_results.empty:
        print(f"  Error: No results found for tier '{model_tier_to_retrain}' in original results.")
        return None

    best_original_row = tier_results.sort_values(by="AUC", ascending=False).iloc[0]
    best_model_name = best_original_row['Model']
    best_original_params = best_original_row['Best_Params']
    original_preprocessor = best_original_row['Preprocessor'] # Get the original preprocessor
    print(f"  Identified best original model: {best_model_name}")
    print(f"  Using best hyperparameters found previously: {best_original_params}")


    # --- Select Top K Features ---
    # Important: We need the *original* feature names before one-hot encoding etc.
    # This requires careful handling based on how get_feature_names_out formats them.
    # We assume the preprocessor stores the original mapping or we derive it.
    # For now, we select based on the *transformed* feature names from SHAP.
    # A more robust solution might involve mapping back to originals if complex interactions aren't used.
    top_k_feature_names_transformed = feature_importance_df['feature'].head(k_features).tolist()
    print(f"  Selected top {k_features} transformed features based on SHAP importance.")


    # --- Prepare Data with ONLY Selected Transformed Features ---
    # This is tricky. We need to apply the *original* preprocessor and then select columns.
    # Alternative: Modify the original feature list *before* prepare_data_for_model.
    # Let's try modifying the feature list first, as it's cleaner if SHAP names map back easily.


    # **Strategy 1: Modify Original Feature List (Simpler if mapping is clear)**
    # This requires mapping SHAP features (e.g., 'num__feature', 'cat__city_NY') back to
    # original features ('feature', 'city'). This is non-trivial.


    # **Strategy 2: Filter AFTER Transformation (More Robust)**
    # We need the full data prepared first, then filter the transformed arrays.
    original_feature_list = model_feature_sets[model_tier_to_retrain]
    X_train_orig_tf, X_test_orig_tf, y_train, y_test, _, feature_names_out_orig = prepare_data_for_model(
        df, original_feature_list, target_col, random_state=random_state
    )


    # Find the indices corresponding to the top k transformed features
    try:
        top_k_indices = [list(feature_names_out_orig).index(f) for f in top_k_feature_names_transformed]
    except ValueError as e:
        print(f"  Error: Could not find all top k features in the preprocessed feature names. {e}")
        # This can happen if feature names changed slightly or SHAP picked an unexpected one.
        # Maybe try intersection:
        top_k_feature_names_transformed = list(set(top_k_feature_names_transformed) & set(feature_names_out_orig))
        top_k_indices = [list(feature_names_out_orig).index(f) for f in top_k_feature_names_transformed]
        print(f"  Using intersection: {len(top_k_indices)} features.")
        if not top_k_indices: return None


    # Filter the transformed data arrays
    X_train_selected_tf = X_train_orig_tf[:, top_k_indices]
    X_test_selected_tf = X_test_orig_tf[:, top_k_indices]
    print(f"  Filtered transformed data to shape: Train {X_train_selected_tf.shape}, Test {X_test_selected_tf.shape}")


    # --- Retrain Best Model with Best Params on Selected Features ---
    print(f"Retraining {best_model_name} with selected features...")
    retrained_model = classifiers[best_model_name] # Get new instance
    retrained_model.set_params(**best_original_params) # Apply best params


    # Apply imbalance handling again (important!)
    scale_pos_weight = None
    if best_model_name in ["XGBoost", "CatBoost", "LightGBM"] and 'class_weight' not in best_original_params:
        neg, pos = np.bincount(y_train)
        if pos > 0: scale_pos_weight = neg / pos
    try:
        if scale_pos_weight is not None:
             if best_model_name == "XGBoost": retrained_model.set_params(scale_pos_weight=scale_pos_weight)
             if best_model_name == "CatBoost": retrained_model.set_params(scale_pos_weight=scale_pos_weight)
             if best_model_name == "LightGBM": retrained_model.set_params(scale_pos_weight=scale_pos_weight)
        elif 'class_weight' in best_original_params and best_original_params['class_weight'] == 'balanced':
             # Ensure balanced is set if it was the best param for RF/LR
             if best_model_name in ["RandomForest", "LogisticRegression"]:
                 retrained_model.set_params(class_weight='balanced')

    except Exception as e:
         print(f"    Warning: Could not set imbalance parameter during retraining. Error: {e}")


    retrained_model.fit(X_train_selected_tf, y_train)


    # --- Evaluate Retrained Model ---
    y_pred_retrained = retrained_model.predict(X_test_selected_tf)
    y_proba_retrained = retrained_model.predict_proba(X_test_selected_tf)[:, 1]

    auc_re = roc_auc_score(y_test, y_proba_retrained)
    acc_re = accuracy_score(y_test, y_pred_retrained)
    prec_re = precision_score(y_test, y_pred_retrained, zero_division=0)
    rec_re = recall_score(y_test, y_pred_retrained, zero_division=0)
    f1_re = f1_score(y_test, y_pred_retrained, zero_division=0)


    print(f"  Retraining complete. AUC: {auc_re:.4f}, Acc: {acc_re:.4f}")

    return {
        "Target": target_col,
        "Retrained_Model": best_model_name,
        "Num_Features": k_features,
        "AUC": auc_re,
        "Accuracy": acc_re,
        "Precision": prec_re,
        "Recall": rec_re,
        "F1": f1_re,
        "Features_Used_Names": top_k_feature_names_transformed # Transformed names
    }

In [ ]:
# ============================ FULL PIPELINE ============================
def run_full_experiment(df, target_cols, feature_sets_dict, classifier_dict, param_grid_dict, cv, n_iter, random_state):
    """
    Runs the complete training, evaluation, and SHAP analysis pipeline for multiple targets and tiers.


    Args:
        df (pd.DataFrame): Input dataframe.
        target_cols (list): List of target column names to iterate through.
        feature_sets_dict (dict): Dictionary mapping tier names to feature lists.
        classifier_dict (dict): Dictionary of base classifiers.
        param_grid_dict (dict): Dictionary of parameter grids for tuning.
        cv (int): Number of CV folds.
        n_iter (int): Number of iterations for RandomizedSearchCV.
        random_state (int): Random seed.


    Returns:
        pd.DataFrame: DataFrame containing all results.
    """
    all_results = []

    for target in target_cols:
        print(f"\n{'='*20} RUNNING EXPERIMENT FOR TARGET: {target} {'='*20}")
        results_for_target = [] # Store results specific to this target run

        # --- Run initial pipeline for all tiers ---
        for tier_name, features in tqdm(feature_sets_dict.items(), desc=f"Tier Progress ({target})"):
            print(f"\n=== Feature Tier: {tier_name} ({target}) ===")
            start_tier_time = time.time()
            try:
                X_train, X_test, y_train, y_test, preprocessor, feature_names_out = prepare_data_for_model(
                    df, features, target, random_state=random_state
                )
            except Exception as e:
                 print(f"  Error during data preparation for tier {tier_name}: {e}. Skipping tier.")
                 continue # Skip to next tier if data prep fails

            if feature_names_out is None:
                 print(f"  Skipping tier {tier_name} due to inability to get feature names.")
                 continue


            tier_results_current_run = [] # Temp list for models within this tier/target run

            for model_name in tqdm(classifier_dict.keys(), desc=f"{tier_name} Models ({target})", leave=False):
                print(f"--- Training {model_name} on {tier_name} ({target})... ---")
                try:
                    model, auc, acc, prec, rec, f1, best_params = train_evaluate_model(
                        X_train, X_test, y_train, y_test, model_name,
                        classifier_dict, param_grid_dict, cv, n_iter, random_state
                    )

                    # === Check model is fitted ===
                    try:
                        check_is_fitted(model)
                        n_features = getattr(model, "n_features_in_", "Unknown")
                        print(f"Fitted {model_name}: {n_features} features")
                    except Exception as e:
                        print(f"Model {model_name} is not fitted: {e}. Skipping save.")
                        continue

                    # === Save model and preprocessor ===
                    model_filename = f"best_model_{target}_{tier_name}_{model_name}_{run_timestamp}.pkl"
                    prep_filename = f"preprocessor_{target}_{tier_name}_{model_name}_{run_timestamp}.pkl"
                    model_path = os.path.join(output_folder, model_filename)
                    prep_path = os.path.join(output_folder, prep_filename)

                    joblib.dump(model, model_path)
                    joblib.dump(preprocessor, prep_path)
                    print(f"Saved model to {model_filename}")
                    print(f"Saved preprocessor to {prep_filename}")

                    # ===========================

                    result_row = {
                        "Target": target, # Add target column identifier
                        "Feature_Tier": tier_name,
                        "Model": model_name,
                        "AUC": auc,
                        "Accuracy": acc,
                        "Precision": prec,
                        "Recall": rec,
                        "F1": f1,
                        "Estimator": model, # Store the fitted model object
                        "Preprocessor": preprocessor, # Store the fitted preprocessor
                        "Feature_Names": feature_names_out, # Store the output feature names
                        "X_train_Transformed": X_train, # Store transformed data for SHAP
                        "Best_Params": best_params
                    }
                    tier_results_current_run.append(result_row)

                    # --- Checkpointing (Improved: include all results so far) ---
                    temp_results_for_checkpoint = all_results + tier_results_current_run
                    checkpoint_data = [{k: v for k, v in row.items() if k not in ["Estimator", "X_train_Transformed", "Feature_Names", "Preprocessor"]} for row in temp_results_for_checkpoint]
                    checkpoint_df = pd.DataFrame(checkpoint_data)
                    checkpoint_df.to_csv(model_results_checkpoints_path, index=False)
                    print(f"Checkpoint saved after {model_name} on {tier_name} ({target}).")

                except Exception as e:
                    print(f"  Error training/evaluating {model_name} on {tier_name} ({target}): {e}")


            # --- Find best model for this tier/target and get SHAP ---
            if tier_results_current_run:
                 tier_results_df = pd.DataFrame(tier_results_current_run).sort_values(by="AUC", ascending=False)
                 top_row_tier = tier_results_df.iloc[0] # Best model for this tier/target

                 print(f"\n--- SHAP Analysis for Best Model in Tier: {tier_name} ({top_row_tier['Model']}) ---")
                 feature_importance_df_tier = get_ranked_shap_features(
                     model=top_row_tier["Estimator"],
                     X_train_transformed=top_row_tier["X_train_Transformed"],
                     feature_names=top_row_tier["Feature_Names"]
                 )
                 # Optionally plot here too if desired, or just calculate importance
                 # plot_shap_summary(top_row_tier["Estimator"], top_row_tier["X_train_Transformed"], top_row_tier["Feature_Names"], f"{tier_name} | {top_row_tier['Model']}")

                 # Store importance DF with the result row for later use (optional, can be large)
                 # Note: This adds a DataFrame to each row, might be better to store separately
                 # top_row_tier["SHAP_Importance_DF"] = feature_importance_df_tier

                 # Append results for this tier/target run to the main list for this target
                 results_for_target.extend(tier_results_current_run)
            else:
                 print(f"  No models successfully trained for tier {tier_name}, skipping SHAP.")

            end_tier_time = time.time()
            print(f"=== Tier {tier_name} ({target}) completed in {(end_tier_time - start_tier_time):.2f} seconds ===")

        # Append all results for this target to the overall results
        all_results.extend(results_for_target)


    # --- Final Results Processing ---
    if not all_results:
         print("\nNo results were generated across all targets.")
         return pd.DataFrame()


    results_df_final = pd.DataFrame(all_results)
    # Sort by Target, Tier, then AUC
    results_df_final = results_df_final.sort_values(by=["Target", "Feature_Tier", "AUC"], ascending=[True, True, False])

    print("\nExperiment Run Complete.")
    return results_df_final


### Run

In [ ]:
# ============================ MAIN EXECUTION ============================

# --- Run the main pipeline for selected targets ---
all_run_results_df = run_full_experiment(
    df=df,
    target_cols=TARGET_COLS_TO_RUN,
    feature_sets_dict=model_feature_sets,
    classifier_dict=classifiers,
    param_grid_dict=param_grids_random,
    cv=CV_FOLDS,
    n_iter=N_ITER_RANDOM_SEARCH,
    random_state=42
)


# --- Display Top Results ---
print("\n" + "="*50)
print("TOP MODEL RESULTS (METRICS ONLY)")
print("="*50)
if not all_run_results_df.empty:
    # Select columns to display (exclude objects)
    display_cols = ["Target", "Feature_Tier", "Model", "AUC", "Accuracy", "Precision", "Recall", "F1", "Best_Params"]
    print(all_run_results_df[display_cols].head(20))

    # Save final metrics summary
    final_metrics_path = os.path.join(output_folder, f"final_model_metrics_summary_{run_timestamp}.csv")
    all_run_results_df[display_cols].to_csv(final_metrics_path, index=False)
    print(f"\nFinal metrics summary saved to: {final_metrics_path}")


else:
    print("No results to display.")

### SHAP & Re-run

In [ ]:
# --- Optional: Get SHAP importance for Overall Best Model (e.g., best AUC on FULL_MODEL for 5y) ---
if not all_run_results_df.empty:
    target_for_shap = "success_label_5y" # Choose the target for detailed SHAP
    tier_for_shap = "FULL_MODEL"       # Choose the tier
    best_overall_row = all_run_results_df[
        (all_run_results_df['Target'] == target_for_shap) &
        (all_run_results_df['Feature_Tier'] == tier_for_shap)
    ].sort_values(by="AUC", ascending=False).iloc[0:1] # Use iloc[0:1] to keep as DataFrame


    if not best_overall_row.empty:
        best_overall_row = best_overall_row.iloc[0] # Now it's a Series
        print(f"\n{'='*20} SHAP IMPORTANCE FOR OVERALL BEST ({target_for_shap}, {tier_for_shap}, {best_overall_row['Model']}) {'='*20}")
        shap_importance_df_best = get_ranked_shap_features(
            model=best_overall_row["Estimator"],
            X_train_transformed=best_overall_row["X_train_Transformed"],
            feature_names=best_overall_row["Feature_Names"]
        )
        if shap_importance_df_best is not None:
            print("\nTop 30 Most Important Features:")
            print(shap_importance_df_best.head(30))
            importance_path_best = os.path.join(output_folder, f"feature_importance_BEST_{best_overall_row['Model']}_{target_for_shap}_{tier_for_shap}.csv")
            shap_importance_df_best.to_csv(importance_path_best, index=False)
            print(f"\nBest model's feature importance saved to: {importance_path_best}")

            # Optionally plot the best one again here
            plot_shap_summary(best_overall_row["Estimator"], best_overall_row["X_train_Transformed"], best_overall_row["Feature_Names"], f"BEST: {tier_for_shap} | {best_overall_row['Model']} ({target_for_shap})")

            # --- Optional: Retrain with Top K Features ---
            # K_FEATURES = 50 # Example: Retrain with top 50 features
            # if shap_importance_df_best.shape[0] >= K_FEATURES:
            #     retrain_results = retrain_with_top_features(
            #         df=df, # Pass original df
            #         target_col=target_for_shap,
            #         original_results_df=all_run_results_df[all_run_results_df['Target'] == target_for_shap], # Pass results for the specific target
            #         feature_importance_df=shap_importance_df_best,
            #         k_features=K_FEATURES,
            #         model_tier_to_retrain=tier_for_shap,
            #         random_state=42
            #     )
            #     if retrain_results:
            #         print("\nRetraining with Top K Features Results:")
            #         print(pd.Series(retrain_results))
            #         # Append retraining results to a separate file or list if needed
            # else:
            #     print(f"\nNot enough features ({shap_importance_df_best.shape[0]}) to select top {K_FEATURES}.")

    else:
        print(f"Could not find results for target '{target_for_shap}' and tier '{tier_for_shap}' to extract SHAP importance.")


In [ ]:
# --- Optional: Save Best Models ---
print("\n" + "="*50)
print("SAVING BEST MODEL PER TIER PER TARGET")
print("="*50)
if not all_run_results_df.empty:
    for target in all_run_results_df["Target"].unique():
         for tier in all_run_results_df["Feature_Tier"].unique():
             tier_target_results = all_run_results_df[
                 (all_run_results_df['Target'] == target) &
                 (all_run_results_df['Feature_Tier'] == tier)
             ]
             if not tier_target_results.empty:
                 best_row = tier_target_results.sort_values(by="AUC", ascending=False).iloc[0]
                 model_filename = f"best_model_{target}_{tier}_{best_row['Model']}_{run_timestamp}.pkl"
                 prep_filename = f"preprocessor_{target}_{tier}_{run_timestamp}.pkl"
                 estimator_path = os.path.join(output_folder, model_filename)
                 preprocessor_path = os.path.join(output_folder, prep_filename)


                 joblib.dump(best_row["Estimator"], estimator_path)
                 joblib.dump(best_row["Preprocessor"], preprocessor_path)
                 print(f"Saved: {model_filename} (AUC: {best_row['AUC']:.4f})")
                 print(f"Saved: {prep_filename}")
             else:
                 print(f"No results found for Target: {target}, Tier: {tier}. Skipping save.")
else:
     print("No results to save models from.")

print("\nPipeline Finished.")